In [63]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
import os
import pickle
%matplotlib inline

In [64]:
def is_nan(x):
    return (x is np.nan or x != x)

In [65]:
#import data
bed_types = ['Studio', '1Bedroom', '2Bedroom', '3Bedroom', '4Bedroom', '5BedroomOrMore']
#bed_types = ['Studio']

attr_df = pd.DataFrame(columns=['RegionName', 'State', 'Metro', 'CountyName', 'SizeRank','BedSize', 'Year','Month', 'Amount'])
attr_df.head()


,RegionName,State,Metro,CountyName,SizeRank,BedSize,Year,Month,Amount


In [66]:
for index, term in enumerate(bed_types):
    #print(term)
    temp_df=pd.read_csv('data/Apartment/City_MedianRentalPrice_{0}.csv'.format(term))
    mask = (temp_df['RegionName'] == 'Seattle') & (temp_df['Metro'] == 'Seattle')
    temp_df = temp_df[mask]
    print(temp_df)
    for index, row in temp_df.iterrows():
        for col in temp_df.columns:
            if col.find("-") > -1 and not is_nan(row[col]):
                split_str = col.split('-')
                temp_amount = row[col]
                new_row = row.iloc[0:5]
                new_row['Year'] = split_str[0]
                new_row['Month'] = split_str[1]
                new_row['Amount'] = temp_amount
                new_row['BedSize'] = term
                
                attr_df = attr_df.append(new_row)

   RegionName State    Metro CountyName  SizeRank  2010-02  2010-03  2010-04  \
18    Seattle    WA  Seattle       King        19      NaN      NaN      NaN   

    2010-05  2010-06   ...     2017-05  2017-06  2017-07  2017-08  2017-09  \
18      NaN      NaN   ...      1512.0   1590.0   1580.0   1555.0   1500.0   

    2017-10  2017-11  2017-12  2018-01  2018-02  
18   1493.0   1438.0   1444.0   1475.0   1450.0  

[1 rows x 102 columns]
   RegionName State    Metro CountyName  SizeRank  2010-02  2010-03  2010-04  \
22    Seattle    WA  Seattle       King        23      NaN      NaN      NaN   

    2010-05  2010-06   ...     2017-05  2017-06  2017-07  2017-08  2017-09  \
22      NaN      NaN   ...      2018.0   2002.0   2041.0   2020.0   2008.0   

    2017-10  2017-11  2017-12  2018-01  2018-02  
22   1985.0   1947.0   1950.0   1975.0   1999.5  

[1 rows x 102 columns]
   RegionName State    Metro CountyName  SizeRank  2010-02  2010-03  2010-04  \
22    Seattle    WA  Seattle       K

In [67]:
attr_df.head()

,RegionName,State,Metro,CountyName,SizeRank,BedSize,Year,Month,Amount
18,Seattle,WA,Seattle,King,19,Studio,2011,03,1097.5
18,Seattle,WA,Seattle,King,19,Studio,2011,04,1095.0
18,Seattle,WA,Seattle,King,19,Studio,2011,05,1025.0
18,Seattle,WA,Seattle,King,19,Studio,2011,06,995.0
18,Seattle,WA,Seattle,King,19,Studio,2011,07,1000.0


In [68]:
studio_df = attr_df.loc[attr_df['BedSize'] == 'Studio']
studio_df['Year'].value_counts()

2015    12
2013    12
2014    12
2017    12
2016    12
2012    12
2011    10
2018     2
Name: Year, dtype: int64

In [69]:
studio_df.shape

(84, 9)

In [70]:
with open('apartment_sale.pkl', 'wb') as picklefile:
    pickle.dump(attr_df, picklefile)

In [71]:
with open("apartment_sale.pkl", 'rb') as picklefile: 
    my_old_data = pickle.load(picklefile)
my_old_data.head()

,RegionName,State,Metro,CountyName,SizeRank,BedSize,Year,Month,Amount
18,Seattle,WA,Seattle,King,19,Studio,2011,03,1097.5
18,Seattle,WA,Seattle,King,19,Studio,2011,04,1095.0
18,Seattle,WA,Seattle,King,19,Studio,2011,05,1025.0
18,Seattle,WA,Seattle,King,19,Studio,2011,06,995.0
18,Seattle,WA,Seattle,King,19,Studio,2011,07,1000.0


# Processing Housing Sale Data From Zilliow

In [72]:
housing_temp_df=pd.read_csv('data/House/Zillow_Sale_Prices_City.csv')
housing_temp_df.columns

Index(['RegionID', 'RegionName', 'StateName', 'SizeRank', '2008-03', '2008-04',
       '2008-05', '2008-06', '2008-07', '2008-08',
       ...
       '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10',
       '2017-11', '2017-12', '2018-01', '2018-02'],
      dtype='object', length=124)

In [73]:
def ProcessingHousingData():
    attr_housing_df = pd.DataFrame(columns=['RegionID', 'RegionName', 'StateName', 'SizeRank', 'Year','Month', 'Amount'])
    temp_df=pd.read_csv('data/House/Zillow_Sale_Prices_City.csv')
    temp_df = temp_df[temp_df['RegionName'] == 'Seattle']
    print(temp_df)
    for index, row in temp_df.iterrows():
        for col in temp_df.columns:
            if col.find("-") > -1 and not is_nan(row[col]):
                split_str = col.split('-')
                temp_amount = row[col]
                new_row = row.iloc[0:4]
                new_row['Year'] = split_str[0]
                new_row['Month'] = split_str[1]
                new_row['Amount'] = temp_amount
                
                attr_housing_df = attr_housing_df.append(new_row)
    return attr_housing_df

In [74]:
zillow_housing_df = ProcessingHousingData()
zillow_housing_df.head()

    RegionID RegionName   StateName  SizeRank   2008-03   2008-04   2008-05  \
17   16037.0    Seattle  Washington        18  421300.0  424000.0  426000.0   

     2008-06   2008-07   2008-08    ...      2017-05   2017-06   2017-07  \
17  422900.0  414400.0  407800.0    ...     621800.0  637900.0  654100.0   

     2017-08   2017-09   2017-10   2017-11   2017-12   2018-01   2018-02  
17  667000.0  674400.0  671400.0  676000.0  677500.0  697000.0  717800.0  

[1 rows x 124 columns]


,RegionID,RegionName,StateName,SizeRank,Year,Month,Amount
17,16037.0,Seattle,Washington,18,2008,03,421300.0
17,16037.0,Seattle,Washington,18,2008,04,424000.0
17,16037.0,Seattle,Washington,18,2008,05,426000.0
17,16037.0,Seattle,Washington,18,2008,06,422900.0
17,16037.0,Seattle,Washington,18,2008,07,414400.0


In [75]:
with open('zillow_housing.pkl', 'wb') as picklefile:
    pickle.dump(zillow_housing_df, picklefile)

In [76]:
with open("zillow_housing.pkl", 'rb') as picklefile: 
    my_old_zillow_housing_df = pickle.load(picklefile)
my_old_zillow_housing_df.head()

,RegionID,RegionName,StateName,SizeRank,Year,Month,Amount
17,16037.0,Seattle,Washington,18,2008,03,421300.0
17,16037.0,Seattle,Washington,18,2008,04,424000.0
17,16037.0,Seattle,Washington,18,2008,05,426000.0
17,16037.0,Seattle,Washington,18,2008,06,422900.0
17,16037.0,Seattle,Washington,18,2008,07,414400.0


In [77]:
my_old_zillow_housing_df.shape

(120, 7)

In [78]:
my_old_zillow_housing_df

,RegionID,RegionName,StateName,SizeRank,Year,Month,Amount
17,16037.0,Seattle,Washington,18,2008,03,421300.0
17,16037.0,Seattle,Washington,18,2008,04,424000.0
17,16037.0,Seattle,Washington,18,2008,05,426000.0
17,16037.0,Seattle,Washington,18,2008,06,422900.0
17,16037.0,Seattle,Washington,18,2008,07,414400.0
17,16037.0,Seattle,Washington,18,2008,08,407800.0
17,16037.0,Seattle,Washington,18,2008,09,404600.0
17,16037.0,Seattle,Washington,18,2008,10,403200.0
17,16037.0,Seattle,Washington,18,2008,11,398200.0
17,16037.0,Seattle,Washington,18,2008,12,399600.0
